In [ ]:
# 创建实例化模型，添加dropout层，和dense层来获得模型的最终输出。 
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.config = BertConfig.from_pretrained('./bert/bert-mini/bert_config.json', output_hidden_states=True)
        self.l1 = BertModel.from_pretrained('./bert/bert-mini/pytorch_model.bin', config=self.config)
        ## 双向LSTM层
        self.bilstm1 = torch.nn.LSTM(512, 64, 1, bidirectional=True)
        # 全连接层
        self.l2 = torch.nn.Linear(128, 64)
        # 激活函数
        self.a1 = torch.nn.ReLU()
        # dropout
        self.l3 = torch.nn.Dropout(0.3)
        self.l4 = torch.nn.Linear(64, 14)
    
    def forward(self, ids, mask, token_type_ids):
        # 将结果送入到各层中去训练
        sequence_output, pooler_output, hidden_states= self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        # [bs, 200, 256]  [bs,256]
        bs = len(sequence_output)
        h12 = hidden_states[-1][:,0].view(1,bs,256)
        h11 = hidden_states[-2][:,0].view(1,bs,256)
        concat_hidden = torch.cat((h12,h11),2)
        x, _ = self.bilstm1(concat_hidden)
        x = self.l2(x.view(bs,128))
        x = self.a1(x)
        x = self.l3(x)
        output = self.l4(x)
        return output

net = BERTClass()
net.to(device)